In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CheckFirst").getOrCreate()
print(spark.sparkContext.appName)
#http://localhost:4040/jobs/

22/03/12 12:30:53 WARN Utils: Your hostname, Anishs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
22/03/12 12:30:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/12 12:30:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


CheckFirst


In [ ]:
# npi bhako case ma network and npi bata check garne....
# npi na bhayeko case ma tyo data lai siteuid bata check garne

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
file1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_10.json"

In [4]:
df = spark.read.json(path=file1)
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
total = df.count()
unique = df.distinct().count()

In [6]:
print(total)
print(unique)

604762
604762


In [12]:
df.select('provider.site_uid').distinct().count()

604762

In [13]:
df.select('provider.npi').distinct().count()

428758

In [14]:
df.select([count(when(col('provider.npi').isNull(),True)).alias('Null NPI')]).show()

+--------+
|Null NPI|
+--------+
|    2819|
+--------+



In [15]:
providerDF = df.select(['provider.first_name','provider.last_name','provider.middle_name','provider.gender','provider.npi']) #provider.* : all

In [16]:
providerDF.select([count(clm) for clm in providerDF]).show()

+-----------------+----------------+------------------+-------------+----------+
|count(first_name)|count(last_name)|count(middle_name)|count(gender)|count(npi)|
+-----------------+----------------+------------------+-------------+----------+
|           498867|          498867|            498867|       498867|    601943|
+-----------------+----------------+------------------+-------------+----------+



In [17]:
npis = df.groupBy("provider.npi").count().orderBy(col('count').desc())
npis.show(10,truncate=False)

+----------+-----+
|npi       |count|
+----------+-----+
|null      |2819 |
|1689612954|76   |
|1295763217|70   |
|1033749940|62   |
|1275988321|60   |
|1326336058|59   |
|1023041159|59   |
|1285975797|46   |
|1922077643|43   |
|1376709535|40   |
+----------+-----+
only showing top 10 rows



In [18]:
fileTest="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/part-00000-12895c9c-1b78-4d49-a923-8bfc058269d6-c000.json"

In [19]:
test = spark.read.json(path=fileTest)
type(test)

pyspark.sql.dataframe.DataFrame

In [26]:
test.select('addresses').printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- office_name: string (nullable = true)
 |    |    |-- phones: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- street_line_1: string (nullable = true)
 |    |    |-- street_line_2: string (nullable = true)
 |    |    |-- zip: string (nullable = true)



In [29]:
test.select('provider.npi').distinct().count()

57573

In [21]:
testnpis = test.groupBy("provider.npi").count().orderBy(col('count').desc())
testnpis.show(10,truncate=False)

+----------+-----+
|npi       |count|
+----------+-----+
|null      |265  |
|1689612954|6    |
|1295763217|6    |
|1396822839|6    |
|1285975797|6    |
|1023041159|6    |
|1275988321|6    |
|1033749940|5    |
|1366499048|5    |
|1326093873|5    |
+----------+-----+
only showing top 10 rows



In [25]:
test.filter(test.provider.npi==1689612954).select("networks.name").show(6,truncate=False)

+------------------------------------------------------------------------------------+
|name                                                                                |
+------------------------------------------------------------------------------------+
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional, Medicare Advantage PPO]|
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional, Medicare Advantage PPO]|
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional, Medicare Advantage PPO]|
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional]                        |
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional]                        |
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional, Medicare Advantage PPO]|
+------------------------------------------------------------------------------------+



In [28]:
test.filter(test.provider.npi==1689612954).select("addresses.street_line_1").show(6,truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+
|street_line_1                                                                                                                      |
+-----------------------------------------------------------------------------------------------------------------------------------+
|[916 N 10th Pl]                                                                                                                    |
|[600 Broadway]                                                                                                                     |
|[18100 NE Union Hill Rd, 3 Cascade, 550 17th Ave, A Floor, 751 NE Blakely Dr, 1801 NW Market St, 7320 216th St SW, 900 Pacific Ave]|
|[751 NE Blakely Dr, 1101 Madison St]                                                                                               |
|[751 NE Blakely Dr, 18100 NE Union Hill Rd]                  

In [30]:
test.filter(test.provider.npi==1033749940).select("networks.name").show(truncate=False)

+------------------------------------------------------------+
|name                                                        |
+------------------------------------------------------------+
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional]|
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional]|
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional]|
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional]|
|[BlueCard PPO Basic, BlueCard PPO/EPO, BlueCard Traditional]|
+------------------------------------------------------------+



In [43]:
#test.filter(test.provider.npi==1033749940).select("addresses.street_line_1",size("addresses.street_line_1")).show(truncate=False)

In [38]:
# test.filter(test.provider.npi==1033749940).select("addresses.street_line_1",size("addresses.street_line_1")).show(truncate=False)
test.filter(test.provider.npi==1033749940).select(size("addresses.street_line_1").alias('street1'),
                                                  size("networks.name").alias('network'),
                                                  size("hospital_affiliations.name").alias('hospital'),
                                                  size("group_affiliations.name").alias('group'),
                                                  'provider.unparsed_name','provider.site_uid'
                                                 ).show(truncate=False)

+-------+-------+--------+-----+---------------------------------+----------------+
|street1|network|hospital|group|unparsed_name                    |site_uid        |
+-------+-------+--------+-----+---------------------------------+----------------+
|100    |3      |0       |0    |Quest Diagnostics                |7405237674506612|
|1      |3      |0       |0    |Quest Diagnostics Bolingbrook PSC|8260482521596443|
|1      |3      |0       |0    |Quest Diagnostics Flossmoor PSC  |3779613306263631|
|1      |3      |0       |0    |Quest Diagnostics Halsted PSC    |6089694029043649|
|1      |3      |0       |0    |Quest Diagnostics Woodridge PSC  |3082571325435363|
+-------+-------+--------+-----+---------------------------------+----------------+



In [41]:
test.filter(test.provider.npi==1689612954).select(size("addresses.street_line_1").alias('street1'),
                                                  size("networks.name").alias('network'),
                                                  size("hospital_affiliations.name").alias('hospital'),
                                                  size("group_affiliations.name").alias('group'),
                                                  'provider.unparsed_name','provider.site_uid','provider.npi'
                                                 ).show(truncate=False)

+-------+-------+--------+-----+--------------------------------------+----------------+----------+
|street1|network|hospital|group|unparsed_name                         |site_uid        |npi       |
+-------+-------+--------+-----+--------------------------------------+----------------+----------+
|1      |4      |0       |0    |Swedish Renton Rehabilitation Services|867282227216975 |1689612954|
|1      |4      |0       |0    |Swedish Hip and Pelvis Center         |8591822720140133|1689612954|
|8      |4      |0       |0    |Swedish Sleep Medicine                |8543672687080976|1689612954|
|2      |3      |0       |0    |Swedish Ci Thoracic Surgery           |8751426381702035|1689612954|
|2      |3      |0       |0    |Swedish Issaquah Otolaryngology       |2066783791014282|1689612954|
|3      |4      |0       |0    |Swedish Head and Neck Surgery         |6311014164257842|1689612954|
+-------+-------+--------+-----+--------------------------------------+----------------+----------+


# Load CHUNK: npi, sort, del-site-uid

In [243]:
fileTest2="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_20.json"
# fileTest2="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_10.json"

In [244]:
test2 = spark.read.json(path=fileTest2)
type(test2)

pyspark.sql.dataframe.DataFrame

In [245]:
total2 = test2.count()
unique2 = test2.distinct().count()

In [246]:
print(total2)
print(unique2)

604762
604762


In [247]:
test2.select('provider.npi').distinct().count()

428758

In [250]:
testnpis2 = test2.groupBy("provider.npi").count().orderBy(col('count').desc())
testnpis2.show(10,truncate=False)

+----------+-----+
|npi       |count|
+----------+-----+
|null      |2819 |
|1689612954|76   |
|1295763217|70   |
|1033749940|62   |
|1275988321|60   |
|1326336058|59   |
|1023041159|59   |
|1285975797|46   |
|1922077643|43   |
|1376709535|40   |
+----------+-----+
only showing top 10 rows



### NEW Schema and Fields: dropping Site_UID

In [255]:
abcd = test2.select(['provider.*',
#     'provider.facility_name','provider.first_name','provider.gender','provider.last_name','provider.suffix',
#                     'provider.license_number','provider.middle_name','provider.npi','provider.provider_type','provider.title','provider.unparsed_name',
                    array_sort('addresses').alias('addresses'),
                    array_sort('networks').alias('networks'),
                    array_sort('hospital_affiliations').alias('hospital_affiliations'),
                    array_sort('group_affiliations').alias('group_affiliations'),
                    array_sort('specialties').alias('specialties')
                   ])
type(abcd)

pyspark.sql.dataframe.DataFrame

In [256]:
abcd = abcd.drop('site_uid')

In [257]:
totalabcd = abcd.count()
uniqueabcd = abcd.distinct().count()

In [258]:
print(totalabcd)
print(uniqueabcd)

604762
517615


In [259]:
demoabcd = abcd.distinct()

In [260]:
demoabcd = demoabcd.withColumn("score",lit(None))
demoabcd = demoabcd.withColumn("scale",lit(None))
demoabcd = demoabcd.withColumn("site_uid",lit(None))

In [262]:
demoabcd1 = demoabcd.select(struct('facility_name', 'first_name','gender','last_name',
                                   'license_number','middle_name','npi','pcp','provider_type','suffix','title','unparsed_name',
                                   struct('scale','score').alias('rating')
                                  ).alias('provider'),'networks','specialties','group_affiliations','hospital_affiliations','addresses')

In [263]:
demoabcd1.printSchema()

root
 |-- provider: struct (nullable = false)
 |    |-- facility_name: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |    |-- site_uid: void (nullable = true)
 |    |-- license_number: string (nullable = true)
 |    |-- middle_name: string (nullable = true)
 |    |-- npi: string (nullable = true)
 |    |-- pcp: boolean (nullable = true)
 |    |-- provider_type: string (nullable = true)
 |    |-- suffix: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- unparsed_name: string (nullable = true)
 |    |-- rating: struct (nullable = false)
 |    |    |-- scale: void (nullable = true)
 |    |    |-- score: void (nullable = true)
 |-- networks: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- tier: string (nullable = true)
 |-- specialties: array (nullable = true)
 |   

In [264]:
demoabcd1.count()

517615

In [265]:
demoabcd1.select('provider.*').show(5,truncate=False)

+--------------------------+----------+------+---------+--------+--------------+-----------+----------+----+-------------+------+-----+--------------------------+------------+
|facility_name             |first_name|gender|last_name|site_uid|license_number|middle_name|npi       |pcp |provider_type|suffix|title|unparsed_name             |rating      |
+--------------------------+----------+------+---------+--------+--------------+-----------+----------+----+-------------+------+-----+--------------------------+------------+
|null                      |Cady      |F     |Elredy   |null    |              |           |1497219349|null|individual   |LAC   |null |Cady Elredy               |{null, null}|
|null                      |Zubin     |M     |Dah      |null    |              |M          |1205194651|null|individual   |LAC   |null |Zubin M Dah               |{null, null}|
|Pandit Rheumatology       |null      |null  |null     |null    |              |null       |1285186049|null|facility    

In [266]:
testnpis3 = demoabcd1.groupBy("provider.npi").count().orderBy(col('count').desc())
testnpis3.show(10,truncate=False)

22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/03/12 17:19:07 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+-----+
|npi       |count|
+----------+-----+
|null      |2397 |
|1689612954|64   |
|1295763217|63   |
|1275988321|56   |
|1326336058|50   |
|1023041159|49   |
|1033749940|39   |
|1376709535|36   |
|1922077643|36   |
|1619209574|33   |
+----------+-----+
only showing top 10 rows



#### <hr />Original NPI's before SiteUID

## <hr /><hr />

In [267]:
demoabcd1.filter(demoabcd1.provider.npi==1689612954).select('addresses.street_line_1',
                                                    size("addresses.address_string").alias('street1'),
                                                  size("networks.name").alias('network'),
                                                  size("hospital_affiliations.name").alias('hospital'),
                                                  size("group_affiliations.name").alias('group'),
                                                  'provider.unparsed_name','provider.site_uid'
                                                   ).show(5,truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+-------+-------+--------+-----+-------------------------------+--------+
|street_line_1                                                                                                                      |street1|network|hospital|group|unparsed_name                  |site_uid|
+-----------------------------------------------------------------------------------------------------------------------------------+-------+-------+--------+-----+-------------------------------+--------+
|[1801 NW Market St, 18100 NE Union Hill Rd, 3 Cascade, 550 17th Ave, 7320 216th St SW, 751 NE Blakely Dr, 900 Pacific Ave, A Floor]|8      |4      |0       |0    |Swedish Sleep Medicine         |null    |
|[18100 NE Union Hill Rd, 751 NE Blakely Dr]                                                                                        |2      |3      |0       |0    |Swedish Issa

In [242]:
#demoabcd1.filter(demoabcd1.provider.npi==1689612954).select('networks','specialties').show(truncate=False)

In [268]:
def writeToFile(df,fName,opDir,mode):
    import os
    if not os.path.exists(opDir):
        os.makedirs(opDir)
        print("Directory Created")       
    
    if mode=='csv':
        df.write.mode('Overwrite').csv(opDir)    
    else:
        print("@@")
        df.write.mode('Overwrite').json(opDir)
        
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(opDir))
    print(list_status)
    file_name = [file.getPath().getName() for file in list_status if file.getPath().getName().startswith('part-')][0]
    print(file_name)
#     print("Renaming "+file_name+" .... "+fName)
    fs.rename(spark._jvm.org.apache.hadoop.fs.Path(opDir+'/'+file_name),spark._jvm.org.apache.hadoop.fs.Path(opDir+'/'+fName))
    for file_name in os.listdir(opDir): #remove part- files if they exist!
        if file_name.startswith('part-'):
            print(file_name)

In [269]:
writeToFile(demoabcd1,'vericred_bcbs_20.json','bcbsOutput','json')

@@


[Lorg.apache.hadoop.fs.FileStatus;@71533ad6
part-00004-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00001-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00003-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00006-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00007-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00002-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00000-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00008-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json
part-00005-d942ec5b-ef11-4ee5-bb88-d4be5b314cf0-c000.json


In [ ]:
#cat bcbs/part-0000*.json > bcbs/bcbs_10.json 

In [ ]:
#{"provider":{"first_name":"Jaime","gender":"M","last_name":"Landman","license_number":"","middle_name":"","npi":"1437268505","provider_type":"individual","suffix":"MD","unparsed_name":"Jaime Landman","rating":{}},"networks":[{"name":"BlueCard PPO Basic","tier":""},{"name":"BlueCard PPO/EPO","tier":""},{"name":"BlueCard Traditional","tier":""}],"specialties":[{"name":"Urology"}],"group_affiliations":[{"name":"Prohealth Partners A Medical Group Inc"},{"name":"UCI Department of Urology"},{"name":"UCI Medical Group"},{"name":"Wu Dept of General Surgery"}],"hospital_affiliations":[{"name":"Long Beach Memorial Medical Center"},{"name":"UC Irvine Medical Center"}],"addresses":[{"address_string":"20350 SW Birch St, Newport Beach, CA 92660","city":"Newport Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"7144567005"}],"state":"CA","street_line_1":"20350 SW Birch St","street_line_2":"","zip":"92660"},{"address_string":"2801 Atlantic Ave, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2801 Atlantic Ave","street_line_2":"","zip":"90806"},{"address_string":"2810 Long Beach Blvd, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2810 Long Beach Blvd","street_line_2":"","zip":"90806"},{"address_string":"4921 Parkview Pl, St Louis, MO 63110","city":"St Louis","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"3143628200"}],"state":"MO","street_line_1":"4921 Parkview Pl","street_line_2":"","zip":"63110"}]}
#{"provider":{"first_name":"Jaime","gender":"M","last_name":"Landman","license_number":"","middle_name":"","npi":"1437268505","provider_type":"individual","suffix":"MD","unparsed_name":"Jaime Landman","rating":{}},"networks":[{"name":"BlueCard PPO Basic","tier":""},{"name":"BlueCard PPO/EPO","tier":""},{"name":"BlueCard Traditional","tier":""}],"specialties":[{"name":"Urology"}],"group_affiliations":[{"name":"Prohealth Partners A Medical Group Inc"},{"name":"UCI Department of Urology"},{"name":"UCI Medical Group"},{"name":"Wu Dept of General Surgery"}],"hospital_affiliations":[{"name":"Long Beach Memorial Medical Center"},{"name":"UC Irvine Medical Center"}],"addresses":[{"address_string":"20350 SW Birch St, Newport Beach, CA 92660","city":"Newport Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"7145092230"}],"state":"CA","street_line_1":"20350 SW Birch St","street_line_2":"","zip":"92660"},{"address_string":"2801 Atlantic Ave, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2801 Atlantic Ave","street_line_2":"","zip":"90806"},{"address_string":"2810 Long Beach Blvd, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2810 Long Beach Blvd","street_line_2":"","zip":"90806"},{"address_string":"4921 Parkview Pl, St Louis, MO 63110","city":"St Louis","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"3143628200"}],"state":"MO","street_line_1":"4921 Parkview Pl","street_line_2":"","zip":"63110"}]}


In [270]:
del abcd
del demoabcd
del demoabcd1

In [271]:
spark.stop()